In [0]:
import math 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
# difference of lasso and ridge regression is that some of the coefficients can be zero i.e. some of the features are 
# completely neglected
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import warnings
warnings.simplefilter('ignore')

In [12]:
cancer = load_breast_cancer()
print(cancer.keys())
cancer_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
print(cancer_df.head(3))
X = cancer.data
Y = cancer.target

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
   mean radius  mean texture  ...  worst symmetry  worst fractal dimension
0        17.99         10.38  ...          0.4601                  0.11890
1        20.57         17.77  ...          0.2750                  0.08902
2        19.69         21.25  ...          0.3613                  0.08758

[3 rows x 30 columns]


In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.3, random_state=31)

In [14]:
elasticNet = ElasticNet()
elasticNet.fit(X_train,y_train)
train_score=elasticNet.score(X_train,y_train)
test_score=elasticNet.score(X_test,y_test)
coeff_used = np.sum(elasticNet.coef_!=0)
print("training score:", train_score )
print("test score: ", test_score)
print("number of features used: ", coeff_used)

training score: 0.6274862888956645
test score:  0.6330011158645811
number of features used:  5


In [15]:
elasticNet_a01 = ElasticNet(alpha=0.01)
elasticNet_a01.fit(X_train,y_train)
train_score_a01=elasticNet_a01.score(X_train,y_train)
test_score_a01=elasticNet_a01.score(X_test,y_test)
coeff_used_a01 = np.sum(elasticNet_a01.coef_!=0)
print("training score for alpha=0.01:", train_score_a01 )
print("test score for alpha =0.01: ", test_score_a01)
print("number of features used: for alpha =0.01:", coeff_used_a01)

"""
alpha=0.1
training score for alpha=0.1: 0.6776346036517484
test score for alpha =0.1:  0.6542523776403809
number of features used: for alpha =0.1: 6
"""
"""
training score for alpha=0.01: 0.7254557013894096
test score for alpha =0.01:  0.6803185095745135
number of features used: for alpha =0.01: 10
"""

training score for alpha=0.01: 0.7254557013894096
test score for alpha =0.01:  0.6803185095745135
number of features used: for alpha =0.01: 10


'\ntraining score for alpha=0.01: 0.7254557013894096\ntest score for alpha =0.01:  0.6803185095745135\nnumber of features used: for alpha =0.01: 10\n'

In [16]:
from sklearn.linear_model import ElasticNetCV
regrElasticNet = ElasticNetCV(cv=10, random_state=0)
regrElasticNet.fit(X_train,y_train)
#ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=None,normalize=False, positive=False, precompute='auto', random_state=0,selection='cyclic', tol=0.0001, verbose=0)
print("Alpha %.2f"%regrElasticNet.alpha_) 
print(regrElasticNet.intercept_) 
train_score_a01=regrElasticNet.score(X_train,y_train)
test_score_a01=regrElasticNet.score(X_test,y_test)
coeff_used_a01 = np.sum(regrElasticNet.coef_!=0)
print("training score for alpha= %.2f"%regrElasticNet.alpha_, train_score_a01 )
print("test score for alpha = %.2f"%regrElasticNet.alpha_,test_score_a01)
print("number of features used: for alpha = %.2f"%regrElasticNet.alpha_, coeff_used_a01)
regrElasticNet.get_params
"""0.7194180187426976
{'alpha': 0.01, 'l1_ratio': 0.0}
train_score =  0.7460510050453089
test_score =  0.693724329299283"""

Alpha 0.41
2.3402176861994604
training score for alpha= 0.41 0.667637333144655
test score for alpha = 0.41 0.6533925634841322
number of features used: for alpha = 0.41 4


"0.7194180187426976\n{'alpha': 0.01, 'l1_ratio': 0.0}\ntrain_score =  0.7460510050453089\ntest_score =  0.693724329299283"

In [17]:
from sklearn.model_selection import GridSearchCV
alpha_range = [0,0.01,0.1,1,10,100]
print(alpha_range)
l1_ratio_range = np.arange(0,1,0.1)
print("l1_ratio",l1_ratio_range)
param_grid = dict(alpha=alpha_range,l1_ratio = l1_ratio_range)
print(param_grid)
grid = GridSearchCV(ElasticNet(), param_grid, cv=5, return_train_score=False)
grid.fit(X_train,y_train)
# examine the best model
print(grid.best_score_)
print(grid.best_params_)
pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
train_score=grid.score(X_train,y_train)
print("train_score = ", train_score)
test_score=grid.score(X_test,y_test)
print("test_score = ", test_score)

[0, 0.01, 0.1, 1, 10, 100]
l1_ratio [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
{'alpha': [0, 0.01, 0.1, 1, 10, 100], 'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])}
0.7194180187426973
{'alpha': 0.01, 'l1_ratio': 0.0}
train_score =  0.7460510050453089
test_score =  0.693724329299283


In [18]:
from sklearn.model_selection import RandomizedSearchCV
alpha_range = [0,0.01,0.1,1,10,100]
l1_ratio_range = np.arange(0,1,0.1)
param_grid = dict(alpha=alpha_range,l1_ratio = l1_ratio_range)
rand = RandomizedSearchCV(ElasticNet(), param_grid, cv=10, return_train_score=False,n_iter=6)
rand.fit(X_train,y_train)
pd.DataFrame(rand.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
print("#########ElasticNet############")
print(rand.best_score_)
print(rand.best_params_)
train_score=rand.score(X_train,y_train)
print("train_score = ", train_score)
test_score=rand.score(X_test,y_test)
print("test_score = ", test_score)
"""
#########ElasticNet############
0.7116765584718803
{'l1_ratio': 0.0, 'alpha': 0.01}
train_score =  0.7460510050453089
test_score =  0.693724329299283
"""

#########ElasticNet############
0.7245413388753128
{'l1_ratio': 0.4, 'alpha': 0}
train_score =  0.7837570558332014
test_score =  0.7313489750964581


"\n#########ElasticNet############\n0.7116765584718803\n{'l1_ratio': 0.0, 'alpha': 0.01}\ntrain_score =  0.7460510050453089\ntest_score =  0.693724329299283\n"

In [19]:
from sklearn.linear_model import Lasso
alpha_range = [0,0.01,0.1,1,10,100]
#l1_ratio_range = np.arange(0,1,0.1)
param_grid = dict(alpha=alpha_range)
rand = RandomizedSearchCV(Lasso(), param_grid, cv=10, return_train_score=False,n_iter=6)
rand.fit(X_train,y_train)
pd.DataFrame(rand.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
print("#########Lasso############")
print(rand.best_score_)
print(rand.best_params_)
train_score=rand.score(X_train,y_train)
print("train_score = ", train_score)
test_score=rand.score(X_test,y_test)
print("test_score = ", test_score)
"""
#########Lasso############
0.7245413388753128
{'alpha': 0}
train_score =  0.7837570558332013
test_score =  0.7313489750964584
"""

#########Lasso############
0.7245413388753128
{'alpha': 0}
train_score =  0.7837570558332014
test_score =  0.7313489750964581


"\n#########Lasso############\n0.7245413388753128\n{'alpha': 0}\ntrain_score =  0.7837570558332013\ntest_score =  0.7313489750964584\n"

In [20]:
from sklearn.linear_model import Ridge
alpha_range = [0,0.01,0.1,1,10,100]
#l1_ratio_range = np.arange(0,1,0.1)
param_grid = dict(alpha=alpha_range)
rand = RandomizedSearchCV(Ridge(), param_grid, cv=10, return_train_score=False,n_iter=6)
rand.fit(X_train,y_train)
pd.DataFrame(rand.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
print("#########Ridge############")
print(rand.best_score_)
print(rand.best_params_)
train_score=rand.score(X_train,y_train)
print("train_score = ", train_score)
test_score=rand.score(X_test,y_test)
print("test_score = ", test_score)
"""
#########Ridge############
0.7257222115149146
{'alpha': 0.01}
train_score =  0.7793449874229254
test_score =  0.7346914231500055
"""

#########Ridge############
0.7257222115148986
{'alpha': 0.01}
train_score =  0.7793449874229257
test_score =  0.7346914231500054


"\n#########Ridge############\n0.7257222115149146\n{'alpha': 0.01}\ntrain_score =  0.7793449874229254\ntest_score =  0.7346914231500055\n"